In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd drive/MyDrive/CS\ 182\ Final\ Project\ /CS\ 182\ Final\ Project/Model

In [ ]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

In [ ]:
from datasets import load_dataset
import re

def read_files(file_names: list):
  dfs = []
  for file in file_names:
    dfs.append(pd.read_csv(f"./../Data Processing/Training data/{file}"))
  df = pd.concat(dfs)
  df.columns = ["q", "a"]
  return df

train_df = read_files(["126.csv", "127.csv", "182.csv", "189_fa.csv", "189_sp.csv"])
dataset = Dataset.from_pandas(train_df)
dataset = dataset.train_test_split(test_size=0.2, seed=20)
train_dataset, val_dataset = dataset["train"], dataset["test"]

# Load the dataset
# dataset_name = 'flax-sentence-embeddings/stackexchange_math_jsonl'
# dataset = load_dataset(dataset_name, name='titlebody_answer')
q_col = 'q'
a_col = 'a'

# Define a function to transform the data
def transform_conversation(example):
  question = example[q_col]
  answer = example[a_col]
  return {'text': f'<s>[INST] {question} [/INST] {answer} </s>'}

# Apply the transformation
train_dataset = train_dataset.map(transform_conversation).select_columns('text')
val_dataset = val_dataset.map(transform_conversation).select_columns('text')

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "openlm-research/open_llama_3b_v2"

# Fine-tuned model name
new_model = "kyle/llama_3b_v2_lora_r64_a16"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./kyle/results"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 8

# Batch size per GPU for evaluation
per_device_eval_batch_size = 8

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 50

# Log every X updates steps
logging_steps = 50

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# # Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# # Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# # Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# # Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# # Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="steps",
)

# # Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

RuntimeError: ignored

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.446, 'learning_rate': 0.0002, 'epoch': 0.41}
{'eval_loss': 1.3213049173355103, 'eval_runtime': 40.6973, 'eval_samples_per_second': 6.069, 'eval_steps_per_second': 0.762, 'epoch': 0.41}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.2963, 'learning_rate': 0.0002, 'epoch': 0.81}
{'eval_loss': 1.287931203842163, 'eval_runtime': 40.7817, 'eval_samples_per_second': 6.057, 'eval_steps_per_second': 0.76, 'epoch': 0.81}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.3054, 'learning_rate': 0.0002, 'epoch': 1.22}
{'eval_loss': 1.2809768915176392, 'eval_runtime': 41.0588, 'eval_samples_per_second': 6.016, 'eval_steps_per_second': 0.755, 'epoch': 1.22}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.2152, 'learning_rate': 0.0002, 'epoch': 1.63}
{'eval_loss': 1.2655614614486694, 'eval_runtime': 40.8618, 'eval_samples_per_second': 6.045, 'eval_steps_per_second': 0.759, 'epoch': 1.63}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
%load_ext tensorboard
%tensorboard --logdir kyle/results/runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-11-27 10:24:55.213327: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 10:24:55.213401: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 10:24:55.213430: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 10:24:56.218222: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorf

In [ ]:
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Explain the concept of stochastic gradient descent (SGD) and how it differs from standard gradient descent in terms of computational efficiency."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] Explain the concept of stochastic gradient descent (SGD) and how it differs from standard gradient descent in terms of computational efficiency. [/INST] Stochastic gradient descent (SGD) is a variant of gradient descent that uses a random sample of the data points instead of the entire dataset to compute the gradient. This approach is computationally efficient because it avoids the need to compute the gradient for all points in the dataset, which can be expensive and time consuming.

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 



In [ ]:
logging

<module 'transformers.utils.logging' from '/usr/local/lib/python3.10/dist-packages/transformers/utils/logging.py'>

##Load Model

In [ ]:
%ls Geovanni/gpt2_lora_r128_a16_lr2e-4

ftgpt-2-medium-test3/  README.txt  results/


In [ ]:
from peft import AutoPeftModelForCausalLM
peft_model_id = "Geovanni/gpt2_lora_r128_a16_lr2e-4/ftgpt-2-medium-test3/"
model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:299: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## Generate Results

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(40)
def read_files(file_names: list, test=False):
  data = 'Training'
  if test:
    data = 'Test'
  dfs = []
  for file in file_names:
    dfs.append(pd.read_csv(f"./../Data Processing/{data} data/{file}"))
  df = pd.concat(dfs)
  df.columns = ["q", "a"]
  return df

train_df = read_files(["126.csv", "127.csv", "189_fa.csv", "189_sp.csv", "182.csv"], test=True)
dataset = Dataset.from_pandas(train_df)

def generateResults(model, tokenizer, dataset, device, index=None):
  prompts = np.random.choice(dataset['q'], 100, replace=False)
  answers = np.random.choice(dataset['a'], 100, replace=False)
  out = None
  results = pd.DataFrame(prompts, columns=["Question"])
  results["Answer"] = answers

  # prompt = "Explain the concept of stochastic gradient descent (SGD) and how it differs from standard gradient descent in terms of computational efficiency."
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200, device=device)
  for i in range(len(prompts)):
    prompts[i] = f"<s>[INST] {prompts[i]} [/INST]"

  # prompts = torch.tensor(np.array(prompts))
  # prompts = prompts.to(device)
  result = pipe(prompts)
  out = None
  for i in range(len(result)):
    if out is None:
      out = [f"{result[i][0]['generated_text']}</s>"]
    else:
      out = np.append(out, [f"{result[i][0]['generated_text']}</s>"], axis=0)

  results["Output"] = out
  if index is not None:
    results.to_csv(f"results{index}.csv")
  else:
    results.to_csv("results.csv")
  return results

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
# model.to('cpu')
with torch.no_grad():
  res = generateResults(model, tokenizer, dataset, device, index=peft_model_id)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

UFuncTypeError: ignored

In [ ]:
res

,Question,Answer,Output
0,Define the covariance matrix and explain the s...,The covariance matrix $\Sigma$ of a random vec...,<s>[INST] Define the covariance matrix and exp...
1,Explain how Definition 1 of jointly Gaussian r...,Definition 1 states that a random vector $\mat...,<s>[INST] Explain how Definition 1 of jointly ...
2,Based on Definition 2 of jointly Gaussian rand...,"According to Definition 2, for random variable...",<s>[INST] Based on Definition 2 of jointly Gau...
3,Given the joint probability density function (...,The joint PDF of a random vector $\mathbf{X}$ ...,<s>[INST] Given the joint probability density ...
4,Define the joint cumulative distribution funct...,The joint cumulative distribution function of ...,<s>[INST] Define the joint cumulative distribu...
...,...,...,...
117,Define the Maximum Likelihood Sequence Estimat...,The Maximum Likelihood Sequence Estimation (ML...,<s>[INST] Define the Maximum Likelihood Sequen...
118,"In the context of the Viterbi Algorithm, expla...","The functions $d_0(x_0)$ and $d_m(x_{m-1}, x_m...",<s>[INST] In the context of the Viterbi Algori...
119,Describe the trellis diagram in the context of...,The trellis diagram is a visual representation...,<s>[INST] Describe the trellis diagram in the ...
120,"When using the Viterbi Algorithm for the ""near...",The edge weights $d_m$ for the trellis diagram...,<s>[INST] When using the Viterbi Algorithm for...


## Get Perplexity

In [ ]:
prompt = ["Explain the concept of stochastic gradient descent (SGD) and how it differs from standard gradient descent in terms of computational efficiency.",
          "Poop d loot abc geo geo string string descent poop Poop d loot abc geo geo string string descent poop Poop d sc",
          "does padding work"]
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
inputs["input_ids"] = inputs["input_ids"].to(device)
loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
ppl = torch.exp(loss)
print(ppl)

tensor(471.0211, device='cuda:0')


In [ ]:
inputs["input_ids"].device

device(type='cpu')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
for input in train_dataset['text']:
  input = tokenizer(input, return_tensors='pt')
  inputs["input_ids"] = inputs["input_ids"].to(device)
  loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
  ppl += torch.exp(loss)
ppl /= len(inputs)

In [ ]:
ppl

tensor(2702.9099, device='cuda:0')

In [ ]:
inputs = tokenizer(train_dataset['text'], return_tensors='pt', padding=True, truncation=True)
inputs["input_ids"] = inputs["input_ids"].to(device)
loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
ppl = torch.exp(loss)
print(ppl)

NameError: ignored

In [ ]:
import torch
from tqdm import tqdm

max_length = model.config.n_positions
stride = 512
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
encodings = tokenizer(val_dataset['text'], return_tensors='pt', padding=True, truncation=True)
# encodings["input_ids"] = encodings["input_ids"].to(device)
seq_len = encodings.input_ids.size(1)

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)

        # loss is calculated using CrossEntropyLoss which averages over valid labels
        # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
        # to the left by 1.
        neg_log_likelihood = outputs.loss

    nlls.append(neg_log_likelihood)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())

  0%|          | 0/1 [00:03<?, ?it/s]


OutOfMemoryError: ignored